In [ ]:
import json
import pandas as pd
import plotly.express as px

import numpy as np

import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from flask_babel import gettext
from plotly.subplots import make_subplots
from scipy import stats
import numpy as np
import math
df_prov_tot = pd.read_csv('static/csv/be-covid-provinces-all.csv')


df_prov_tot['POSITIVE_RATE'] = df_prov_tot['CASES'] / df_prov_tot['TESTS_ALL']
df_prov_tot['TESTING_RATE'] = df_prov_tot['TESTS_ALL'] * 100000 / df_prov_tot['POP']

from datetime import datetime, date, timedelta


# Cases in each province

In [ ]:
def cases_prov(prov=None):
    df_cases = df_prov_tot

    if prov != 'All':
        df_cases = df_cases[df_cases['PROVINCE'] == prov]

    df = df_cases.groupby(['DATE']).agg({'CASES':'sum','TESTS_ALL':'sum','TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})

    x2 = df.index
    y2 = df.CASES

    df.index = pd.to_datetime(df.index)
    dw = df.index.dayofweek
    colors_map = {0: '#fccde5', 1: '#8dd3c7', 2: '#b3de69', 3: '#bebada', 4: '#fb8072', 5: '#fdb462', 6: '#80b1d3',
                  7: 'lightgrey'}
    colors = [colors_map[i] for i in dw]

    
    fig = go.Figure(data=[
        go.Bar(x=x2, y=y2, name='bars', legendgroup='group2', showlegend=True, marker_color=colors),
        go.Scatter(x=x2[:-2], y=y2[:-2].rolling(7,center=True).mean(), showlegend=True, name=gettext('avg'),legendgroup='avg 7 days', marker_color="red")
    ])
    
    fig.add_vline(x=(datetime.today()-timedelta(days=4)).strftime('%Y-%m-%d'), line_width=3, line_dash="dash", line_color="green")

    
    
    start = '2021-01-01' 
    fig.update_layout(yaxis_range=[0,max(y2[x2 >=start])])
    fig.update_layout(xaxis_range=[start, (datetime.today()).strftime('%Y-%m-%d')])
    fig.update_layout(template="plotly_white", height=300, margin=dict(l=0, r=0, t=30, b=0),title=prov)

    return fig


def cases_per_provinces():
    #prov = ['All','Liège', 'Brussels','Antwerpen','Namur', 'Luxembourg', 'Hainaut' , 'BrabantWallon', 'VlaamsBrabant', 'OostVlaanderen',
    #    'WestVlaanderen', 'Limburg',]
    prov = ['All']
    n_cols = 2
    n_rows = math.ceil(1.0 * len(prov)/ n_cols)

    idx = 0
    for prov_name in prov:
            fig = cases_prov(prov=prov_name)
            
            fig.show()



cases_per_provinces()

In [ ]:
def plot(df, column_name, title):
    bars = []
    provinces = df.PROVINCE.unique()
    cols = px.colors.qualitative.Plotly
    i = 0
    for p in provinces:
        c = cols[i % len(cols)]
        df_p = df.loc[df['PROVINCE'] == p]
        bars.append(go.Scatter(
            x=df_p.DATE[:-4],
            y=df_p[column_name][:-4].rolling(7, center=True).mean(),
            marker_color=c,
            name=p
        ))

        bars.append(go.Scatter(
            x=df_p.DATE,
            y=df_p[column_name],
            mode='markers',
            marker_color=c,
            name=p, visible='legendonly',
        ))
        i += 1
    fig = go.Figure(data=bars, layout=go.Layout(barmode='group'), )
    fig.update_layout(template="plotly_white", height=500, margin=dict(l=0, r=0, t=30, b=0), title=title)
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])
    return fig

def avg_cases_provinces():
    return plot(df_prov_tot,'CASES',"Cases avg 7 days")

avg_cases_provinces().show()

# Cases in each region

In [ ]:
def plot(df, column_name, title):
    bars = []
    regions = df.REGION.unique()
    cols = px.colors.qualitative.Plotly
    df = df.groupby(['DATE','REGION']).sum().reset_index()
    i = 0
    for p in regions:
        c = cols[i % len(cols)]
        df_p = df.loc[df['REGION'] == p]
        bars.append(go.Scatter(
            x=df_p.DATE[:-4],
            y=df_p[column_name][:-4].rolling(7, center=True).mean(),
            marker_color=c,
            name=p
        ))

        bars.append(go.Scatter(
            x=df_p.DATE,
            y=df_p[column_name],
            mode='markers',
            marker_color=c,
            name=p, visible='legendonly',
        ))
        i += 1
    fig = go.Figure(data=bars, layout=go.Layout(barmode='group'), )
    fig.update_layout(template="plotly_white", height=500, margin=dict(l=0, r=0, t=30, b=0), title=title)
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])
    return fig

def avg_cases_provinces():
    return plot(df_prov_tot,'CASES',"Cases avg 7 days")

avg_cases_provinces().show()

# Positive Rate

In [ ]:
def plot_positive_rate():
    """
    plot of the ration cases over testing everyday
    """
    
    df = df_prov_tot.groupby(['DATE']).agg({'CASES':'sum','TESTS_ALL':'sum','TESTS_ALL_POS':'sum','TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})


    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index[:-4], y=100 * df.TESTS_ALL_POS[:-4].rolling(7,center=True).mean() / df.TESTS_ALL.rolling(7, center=True).mean(),
                             mode='lines',
                             name='avg',
                             marker_color="blue", legendgroup="two", showlegend=True))

    fig.add_trace(go.Scatter(x=df.index, y=100 * df.TESTS_ALL_POS[:-4] / df.TESTS_ALL[:-4],
                             mode='markers',
                             name='Pos %',
                             marker_color="blue", legendgroup="twoa", showlegend=True))

    fig.update_layout(xaxis_title='Day',
                      yaxis_title='Pos %', title="Positive rate %")
    fig.update_layout(template="plotly_white")
    
    start = '2021-01-01' 
    fig.update_layout(yaxis_range=[0,25])
    fig.update_layout(xaxis_range=[start, (datetime.today()).strftime('%Y-%m-%d')])

    fig.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])

    return fig
plot_positive_rate()

In [ ]:
df_region = df_prov_tot.groupby([df_prov_tot.DATE, df_prov_tot.REGION]).agg(
    {'CASES': ['sum'], 'TESTS_ALL': ['sum'], 'TESTS_ALL_POS': ['sum'], 'NEW_IN': ['sum']}).reset_index()
df_region.columns = df_region.columns.get_level_values(0)


def trace(fig, region, log=False, div=1, mul=1, col="blue"):
    fig.add_trace(go.Scatter(name=region,
                             x=df_region[df_region.REGION == region].DATE[:-4],
                             y=df_region[df_region.REGION == region].CASES[:-4].rolling(7, center=True).mean() / div * mul,
                             marker_color=col, legendgroup=region + "mean", showlegend=True))

    fig.add_trace(go.Bar(name=region,
                         x=df_region[df_region.REGION == region].DATE[:-4],
                         y=df_region[df_region.REGION == region].CASES[:-4] / div * mul, legendgroup=region, marker_color=col,
                         visible='legendonly',
                         showlegend=True))

    fig.add_trace(go.Bar(name=region+" not consolidated",
                         x=df_region[df_region.REGION == region].DATE[-4:],
                         y=df_region[df_region.REGION == region].CASES[-4:] / div * mul, legendgroup=region, marker_color="grey",
                         visible='legendonly',
                         showlegend=True))
    
def plot_relative_cases():
    fig = go.Figure()
    brux_pop = 1218255
    wall_pop = 3645243
    flanders_pop = 6629143

    trace(fig, 'Brussels', div=brux_pop, mul=100000, col=px.colors.qualitative.Plotly[0])
    trace(fig, 'Flanders', div=flanders_pop, mul=100000, col=px.colors.qualitative.Plotly[1])
    trace(fig, 'Wallonia', div=wall_pop, mul=100000, col=px.colors.qualitative.Plotly[2])

    fig.update_layout(template="plotly_white", title="Number of cases/100000 habitants")
    #fig.update_layout(xaxis_range=['2021-02-01', datetime.today().strftime('%Y-%m-%d')])
    #fig.update_layout(yaxis_range=[0, 70])

    fig.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])
    return fig

plot_relative_cases()

In [ ]:
def positive_rate_region():
    df_pos = df_prov_tot.groupby([df_prov_tot.DATE,df_prov_tot.REGION]).agg({'TESTS_ALL': ['sum'],'TESTS_ALL_POS': ['sum']}).reset_index()
    df_pos.columns = df_pos.columns.get_level_values(0)
    df_pos['POSITIVE_RATE'] = 100*df_pos['TESTS_ALL_POS']/df_pos['TESTS_ALL']
    df_pos.sort_values(by=['DATE'], inplace=True, ascending=True)


    fig = go.Figure()
    #fig = px.line(x=df_pos.DATE,y=df_pos.POSITIVE_RATE,color=df.REGION)
    regions = ['Brussels','Flanders','Wallonia']
    plots = []
    colors = px.colors.qualitative.Plotly
    i = 0
    for r in regions:
        c = colors[i]
        dfr = df_pos[df_pos.REGION == r]
        plot = go.Scatter(x=dfr['DATE'][:-4], y=dfr['POSITIVE_RATE'][:-4].rolling(7,center=True).mean(),mode='lines',name=r,marker_color=c)
        plots.append(plot)
        plot = go.Scatter(x=dfr['DATE'], y=dfr['POSITIVE_RATE'],name=r+' avg',mode='markers',marker_color=c)
        plots.append(plot)

        i += 1
    fig = go.Figure(data=plots, layout=go.Layout(barmode='group'))
    fig.update_layout(template="plotly_white", title="Positive Rate (%)")
    fig.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])
    return fig
positive_rate_region().show()

In [ ]:
def plot_ratio(df, column_name1,column_name2, title):
    bars = []
    provinces = sorted(df.PROVINCE.unique())
    cols = px.colors.qualitative.Plotly
    i = 0
    for p in provinces:
        c = cols[i % len(cols)]
        df_p = df.loc[df['PROVINCE'] == p]
        bars.append(go.Scatter(
            x=df_p.DATE,
            y=100* (df_p[column_name1].rolling(7,center=True).sum()/df_p[column_name2].rolling(7,center=True).sum()),
            name=p,
            mode='lines',
            marker_color=c
        ))
        bars.append(go.Scatter(
            x=df_p.DATE,
            y=100* df_p[column_name1]/df_p[column_name2],
            name=p,
            mode='markers',
            marker_color=c,visible='legendonly',
        ))
        i += 1


    fig = go.Figure(data=bars,layout=go.Layout(barmode='group'),)
    fig.update_layout(template="plotly_white", height=500,margin=dict(l=0, r=0, t=30, b=0), title=title)
    fig.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])
    return fig




def avg_positive_rate_provinces():
    return plot_ratio(df_prov_tot, 'TESTS_ALL_POS','TESTS_ALL', "Positive rate % (positive tests/ all tests) avg 7 days")

avg_positive_rate_provinces().show()

# Tests

In [ ]:
def plot_tests():
    """
    plot of the ration cases over testing everyday
    """
    
    df = df_prov_tot.groupby(['DATE']).agg({'CASES':'sum','TESTS_ALL':'sum','TESTS_ALL_POS':'sum','TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})


    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index[:-4], y= df.TESTS_ALL.rolling(7, center=True).mean(),
                             mode='lines',
                             name='avg',
                             marker_color="blue", legendgroup="two", showlegend=True))

    fig.add_trace(go.Scatter(x=df.index[:-4], y=df.TESTS_ALL,
                             mode='markers',
                             name='daily',
                             marker_color="blue", legendgroup="twoa", showlegend=True))

    fig.update_layout(xaxis_title=gettext('Day'),
                      yaxis_title=gettext('Number of'), title=gettext("Tests all % (7 days avg)"))
    fig.update_layout(template="plotly_white")
    
    start = '2021-01-01' 
    fig.update_layout(xaxis_range=[start, (datetime.today()).strftime('%Y-%m-%d')])

    fig.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])

    return fig
plot_tests()

In [ ]:
def avg_testing_provinces():
    return plot(df_prov_tot,'TESTS_ALL',"Testing avg 7 days")

avg_testing_provinces().show()